In [1]:
from urllib.request import urlopen

from bs4 import BeautifulSoup
from statistics import mean 
import pandas as pd
import re
from datetime import datetime,timedelta 


import requests


import warnings
warnings.filterwarnings("ignore")

In [2]:
#дата сегодня
date_now=datetime.today().date()
day_now=date_now.day #<--- день сегодня
date_tomorrow=date_now + timedelta(days=1)
day_tomorrow=(date_now + timedelta(days=1)).day #<--- день завтра
day_after_tomorrow=(date_now + timedelta(days=2)).day #<--- день послезавтра
day_4=(date_now + timedelta(days=3)).day #<--- 4-ый
day_5=(date_now + timedelta(days=4)).day #<--- 5-ый
day_6=(date_now + timedelta(days=5)).day #<--- 6-ый
day_7=(date_now + timedelta(days=6)).day #<--- 7-ый


date_after_tomorrow=(date_now + timedelta(days=2))
date_4=(date_now + timedelta(days=3)) #<--- 4-ый
date_5=(date_now + timedelta(days=4)) #<--- 5-ый
date_6=(date_now + timedelta(days=5)) #<--- 6-ый
date_7=(date_now + timedelta(days=6)) #<--- 7-ый
date_8=(date_now + timedelta(days=7)) #<--- 8-ый

#время сегодня
time_now=datetime.today().time().hour

## МЕТЕО-ИНФО

In [3]:
table_meteo_info_0=pd.read_html('https://meteoinfo.ru/forecasts/russia/moscow-area/moscow')

#перевод в датафреймы
meteo_info_df=pd.DataFrame(table_meteo_info_0[4])


meteo_info_df.head(20)

meteo_info_df.drop([1],axis=0,inplace=True)
meteo_info_df.drop([6],axis=0,inplace=True)

meteo_info_df=meteo_info_df.T
meteo_info_df.columns=['date','temp_d','weather','wind_d','press_d','temp_night','weater_n','wind_night','press_n']
meteo_info_df.drop([0],axis=0,inplace=True)

#делим датасет
meteo_info_d=meteo_info_df[['date','temp_d','wind_d','press_d','weather']]
meteo_info_n=meteo_info_df[['date','temp_night','wind_night','press_n','weather']]


meteo_info_n.drop([1],axis=0,inplace=True)

meteo_info_d=meteo_info_d.reset_index(drop=True)
meteo_info_n=meteo_info_n.reset_index(drop=True)


#DAY 
temp_day=meteo_info_d['temp_d'].str.rsplit('..',expand=True)
temp_day[1]=temp_day[1].str.replace('°','')

temp_day[0]=temp_day[0].astype('float')
temp_day[1]=temp_day[1].astype('float')

meteo_info_d['press_d']=meteo_info_d['press_d'].astype('int')

temp_day.columns=['t_day_down','t_day_up']

#NIGHT 
temp_night=meteo_info_n['temp_night'].str.rsplit('..',expand=True)
temp_night[1]=temp_night[1].str.replace('°','')

temp_night[0]=temp_night[0].astype('float')
temp_night[1]=temp_night[1].astype('float')

meteo_info_n['press_n']=meteo_info_n['press_n'].astype('int')

temp_night.columns=['t_night_down','t_night_up']


def wind_redact(data,column):  
    data[column]=data[column].str.replace(' ','')

    k=0
    
    
    data[column]=data[column].str.replace('пригрозе','местами')
    
    
    for i in range(0,len(data[column].index)):
            wind_stroka=data[column][i] 
            if wind_stroka.find('местами') != -1:
                k +=1

    if k !=0:
        wind=data[column].str.rsplit('местами',expand=True).fillna(0)
        
        if column=='wind_d':
            wind.columns=['wind','wind_max']
        if column=='wind_night':
            wind.columns=['wind','wind_max']
        
        wind=pd.DataFrame(wind)
        
        for i in range(0,len(wind['wind_max'].index)):
            wind_str=''
            new_str=''
            wind_str=str(wind['wind_max'][i]) 
            if wind_str.find('-') == -1:
                new_str=re.sub('\D','',wind_str)
                wind['wind_max'][i] = new_str
            if wind_str.find('-') != -1:   
                sep=re.split('-',wind_str)
                new_str=re.sub('\D','',sep[1])
                wind['wind_max'][i] = new_str
    else:
        wind=data[column]
        wind=pd.DataFrame(wind)
        wind['wind_max']=0
        wind.columns=['wind','wind_max']
        
    display(wind)    

    wind['wind']=wind['wind'].str.replace('Ю','')
    wind['wind']=wind['wind'].str.replace('С','')
    wind['wind']=wind['wind'].str.replace('З','')
    wind['wind']=wind['wind'].str.replace('В','')

    wind['wind']=wind['wind'].str.replace('Ю-B','')
    wind['wind']=wind['wind'].str.replace('Ю-З','')

    wind['wind']=wind['wind'].str.replace('С-B','')
    wind['wind']=wind['wind'].str.replace('С-З','')

        #иностранные литералы!

    wind['wind']=wind['wind'].str.replace('C','')
    wind['wind']=wind['wind'].str.replace('B','')
    wind['wind']=wind['wind'].str.replace('Ю-B','')

    wind['wind']=wind['wind'].str.replace('C-B','')
    wind['wind']=wind['wind'].str.replace('С-B','')
    wind['wind']=wind['wind'].str.replace('C-B','')
    wind['wind']=wind['wind'].str.replace('C-З','')

    wind['wind']=wind['wind'].str.replace('слабый','0')
    wind['wind']=wind['wind'].str.replace('штиль','0')
    wind['wind']=wind['wind'].str.replace('переменныхнаправлений','') 
    wind['wind']=wind['wind'].str.replace('небольшойкратковременныйдождь','')
    wind['wind']=wind['wind'].str.replace('спереходомна','') 
    wind['wind']=wind['wind'].str.replace('спорывами','') 
    
    
    wind['wind']=wind['wind'].str.replace('переменный','') 
    wind['wind']=wind['wind'].str.replace('м/c','')
    wind['wind']=wind['wind'].str.replace('м/с','')
    wind['wind']=wind['wind'].str.replace(',','')

    wind_source=wind['wind'].str.rsplit('-',expand=True)

    #обрабатываем первый столбец
    wind_source=wind_source.fillna('')
    wind_source=wind_source.replace('','-00')
    
    if wind_source.shape[1]==3:
        
        wind_source.columns=['a','b','c']
        data_index=wind_source.index.to_list()
        
        wind_source_copy=pd.DataFrame(columns=['a','b','c'],index=data_index)


        for i in range(0,len(wind_source.index)):

            if wind_source['a'][i]!= '-00' and wind_source['b'][i]!= '-00':
                wind_source_copy['a'][i]=wind_source['a'][i]
                wind_source_copy['b'][i]=wind_source['b'][i]

            if wind_source['a'][i] !='-00' and wind_source['b'][i]=='-00' and wind_source['c'][i] !='-00':
                wind_source_copy['a'][i]=wind_source['a'][i]
                wind_source_copy['b'][i]=wind_source['c'][i]

            if wind_source['a'][i] =='-00' and wind_source['b'][i]!='-00' and wind_source['c'][i] !='-00':
                wind_source_copy['a'][i]=wind_source['b'][i]
                wind_source_copy['b'][i]=wind_source['c'][i]    

            if wind_source['a'][i] =='-00' and wind_source['b'][i] =='-00' and wind_source['c'][i] !='-00':
                wind_source_copy['a'][i]=wind_source['a'][i]
                wind_source_copy['b'][i]=wind_source['b'][i]

            if wind_source['a'][i] =='-00' and wind_source['b'][i] !='-00' and wind_source['c'][i] =='-00':
                wind_source_copy['a'][i]=wind_source['b'][i]
                wind_source_copy['b'][i]=wind_source['b'][i]

            if wind_source['a'][i] !='-00' and wind_source['b'][i] =='-00' and wind_source['c'][i]=='-00':
                wind_source_copy['a'][i]=wind_source['a'][i]
                wind_source_copy['b'][i]=wind_source['a'][i] 
                
    if wind_source.shape[1]==2: 
        
        wind_source.columns=['a','b']
        data_index=wind_source.index.to_list()
        wind_source_copy=pd.DataFrame(columns=['a','b'],index=data_index)
        
        for i in range(0,len(wind_source.index)):
        
            if wind_source['a'][i]!= '-00' and wind_source['b'][i]!= '-00':
                    wind_source_copy['a'][i]=wind_source['a'][i]
                    wind_source_copy['b'][i]=wind_source['b'][i]

            if wind_source['a'][i]== '-00' and wind_source['b'][i]== '-00':
                    wind_source_copy['a'][i]=0
                    wind_source_copy['b'][i]=0

            if wind_source['a'][i] !='-00' and wind_source['b'][i]=='-00':
                    wind_source_copy['a'][i]=wind_source['a'][i]
                    wind_source_copy['b'][i]=wind_source['a'][i]

            if wind_source['a'][i] =='-00' and wind_source['b'][i]!='-00':
                    wind_source_copy['a'][i]=wind_source['b'][i]
                    wind_source_copy['b'][i]=wind_source['b'][i]                
        

    wind_source_fin=wind_source_copy[['a','b']]
    wind_source_fin.columns=['wind_d_down','wind_d_up']
    wind_source_fin['wind_max']=wind['wind_max']
    wind_source_fin['wind_d_down']= wind_source_fin['wind_d_down'].astype('int')
    wind_source_fin['wind_d_up']= wind_source_fin['wind_d_up'].astype('int')
    wind_source_fin['wind_max']= wind_source_fin['wind_max'].astype('int')  
    

    return wind_source_fin

wind_day=wind_redact(meteo_info_d,'wind_d')
wind_night=wind_redact(meteo_info_n,'wind_night')


meteo_info_day_fin=pd.concat([meteo_info_d['date'],temp_day,meteo_info_d['press_d'],wind_day,meteo_info_d['weather']],axis=1,ignore_index=True)
meteo_info_night_fin=pd.concat([meteo_info_n['date'],temp_night,meteo_info_n['press_n'],wind_night,meteo_info_n['weather']],axis=1,ignore_index=True)

meteo_info_day_fin.columns=['date','temp_d_min','temp_d_max','press_d','wind_d_down','wind_d_up','wind_d_max','weather']
meteo_info_night_fin.columns=['date','temp_n_min','temp_n_max','press_n','wind_n_down','wind_n_up','wind_n_max','weather']

#средние
meteo_info_day_fin['t_mean']=(meteo_info_day_fin['temp_d_min']+meteo_info_day_fin['temp_d_max'])/2
meteo_info_day_fin['wind_mean']=(meteo_info_day_fin['wind_d_down']+meteo_info_day_fin['wind_d_up'])/2

meteo_info_night_fin['t_mean']=(meteo_info_night_fin['temp_n_min']+meteo_info_night_fin['temp_n_max'])/2
meteo_info_night_fin['wind_mean']=(meteo_info_night_fin['wind_n_up']+meteo_info_night_fin['wind_n_down'])/2

,wind,wind_max
0,"C,C-B4-9м/c",0
1,C-B5-10м/c,0
2,C-B5-10м/c,0
3,"C-B,B4-9м/c",0
4,слабый,0
5,слабый,0
6,слабый,0


,wind,wind_max
0,C-B5-10м/c,0
1,C-B5-10м/c,0
2,"C-B,B4-9м/c",0
3,слабый,0
4,слабый,0
5,слабый,0


In [4]:
meteo_info_night_fin

,date,temp_n_min,temp_n_max,press_n,wind_n_down,wind_n_up,wind_n_max,weather,t_mean,wind_mean
0,Четверг6 апреля,2.0,4.0,754,5,10,0,Переменная облачность. Без осадков,3.0,7.5
1,Пятница7 апреля,0.0,2.0,756,5,10,0,Переменная облачность. Преимущественно без оса...,1.0,7.5
2,Суббота8 апреля,0.0,2.0,758,4,9,0,Переменная облачность. Без осадков,1.0,6.5
3,Воскресенье9 апреля,1.0,6.0,757,0,0,0,Переменная облачность. Без осадков,3.5,0.0
4,Понедельник10 апреля,1.0,6.0,757,0,0,0,Переменная облачность. Без осадков,3.5,0.0
5,Вторник11 апреля,1.0,6.0,757,0,0,0,Облачно с прояснениями. Без осадков,3.5,0.0


In [5]:
meteo_info_day_fin

,date,temp_d_min,temp_d_max,press_d,wind_d_down,wind_d_up,wind_d_max,weather,t_mean,wind_mean
0,Среда5 апреля,12.0,14.0,752,4,9,0,Облачно с прояснениями. Преимущественно без ос...,13.0,6.5
1,Четверг6 апреля,10.0,12.0,755,5,10,0,Переменная облачность. Без осадков,11.0,7.5
2,Пятница7 апреля,9.0,11.0,757,5,10,0,Переменная облачность. Преимущественно без оса...,10.0,7.5
3,Суббота8 апреля,11.0,13.0,758,4,9,0,Переменная облачность. Без осадков,12.0,6.5
4,Воскресенье9 апреля,10.0,15.0,758,0,0,0,Переменная облачность. Без осадков,12.5,0.0
5,Понедельник10 апреля,10.0,15.0,757,0,0,0,Переменная облачность. Без осадков,12.5,0.0
6,Вторник11 апреля,10.0,15.0,756,0,0,0,Облачно с прояснениями. Без осадков,12.5,0.0
